# Introduction

The requirements for undergraduate CS certificate can be found at https://guide.wisc.edu/undergraduate/letters-science/computer-sciences/computer-sciences-ba/#requirementstext. 

# Mathematical Model

### Notations

$T$ is the max number of maximum number of semesters (e.g., $T = 8$ means the entire 4-year college).

$ t\in \{1, \cdots, T\}$ stands for a specific semester (e.g., $t = 2$ means the second semester or the Spring semester in the first year).

$C$ is the set of classes (e.g., $C = \{ \text{CS 200}, \text{CS 300}, \text{CS 524}, \cdots \}$).

$c \in C$ stands for a specific class (e.g., $c = \text{CS 524}$)

### Decision variables

$x[t, c]$ is a Boolean variable to denote whether to take the class $c \in C$ at semester $t \in \{1, \cdots, T\}$. 

For example, if a student takes CS 524 on the thrid semester, then $x[3, \text{CS 524}]= 1$


### Prerequisite constraint

To take the class $c$, students may need to take $c'$ in the previous semesters, we can encode such prerequisite using the following constraint: 

$$x[t, c] \le \sum_{i=1}^{t - 1} x[i, c'] $$


For example, CS 524 has the prerequisite similar to "(CS 200 or 300) and (MATH 340 or 341)", we can encode it with the constraint:

$$x[t, \text{CS 524}] \le \sum_{i=1}^{t - 1} x[i, \text{CS 200}] + \sum_{i=1}^{t - 1} x[i, \text{CS 300}] $$

$$x[t, \text{CS 524}] \le \sum_{i=1}^{t - 1} x[i, \text{CS 340}] + \sum_{i=1}^{t - 1} x[i, \text{CS 341}] $$


### Graduation requirement

In order to meet the graduation requirement, some class $c$ must be taken, so we have the constraint:

$$\sum_{t=1}^{T} x[t, c] \ge 1$$
    
There are other kinds of graduation requirements, such as taking $k$ coureses from a list of $C$, we can encode such requirements using the following constraint:

$$\sum_{t=1}^{T} \sum_{c \in C} x[t, c] \ge k$$
    
### Max credits

For undergraduate, the number of credits taken in a given semester cannot exceeds a certain number, so we need to ensure that

$$\sum_{c \in C} x[t, c] \cdot \text{credit}(c) \le \text{max_credit}, \quad \text{for all semester } t$$

### No retaking

Generally, we don't want to take the same class multiple times, so we have the following constraint to ensure that a class is taken at most once.

$$\sum_{t=1}^{T} x[t, c] \le 1, \quad \text{for all class } c$$

### Objective 

A naive objective would be to minimize the sum of $x$: 

$$\min \sum_{i=t}^{T} \sum_{c \in C} x[t, c]$$

But this would lead to many equally good solutions. To avoid this, we add a weight $t$ to the class $c$ if its taken at semester $t$

$$\min \sum_{i=t}^{T} \sum_{c \in C} x[t, c] \cdot t$$

This avoid students procrastinating class to later semesters. 

# Data Preparation

We first load the json file obtained from the UW-Madison [Course Search & Enrollment website](enroll.wisc.edu), and extract all the CS courses.

In [20]:
using JSON

raw_data = JSON.parsefile("data/1214-spring-2021.json")["hits"]

cls_data = Dict(
    Symbol(c["courseDesignation"]) => 
    c for c in raw_data 
    if c["subject"]["shortDescription"] in ["COMP SCI", "MATH"] && parse(Int, c["catalogSort"]) <= 700
)

cls = keys(cls_data)
n_cls = length(cls_data)

cs_cls_str = join(sort([String(e) for e in cls]), ", ")
println("We have ", string(n_cls), " undergraduate courses in total: ", cs_cls_str)

We have 116 undergraduate courses in total: COMP SCI 200, COMP SCI 220, COMP SCI 240, COMP SCI 252, COMP SCI 270, COMP SCI 298, COMP SCI 300, COMP SCI 304, COMP SCI 310, COMP SCI 319, COMP SCI 320, COMP SCI 352, COMP SCI 354, COMP SCI 368, COMP SCI 369, COMP SCI 400, COMP SCI 402, COMP SCI 407, COMP SCI 412, COMP SCI 425, COMP SCI 435, COMP SCI 475, COMP SCI 502, COMP SCI 506, COMP SCI 513, COMP SCI 514, COMP SCI 520, COMP SCI 524, COMP SCI 525, COMP SCI 532, COMP SCI 533, COMP SCI 534, COMP SCI 536, COMP SCI 537, COMP SCI 538, COMP SCI 539, COMP SCI 540, COMP SCI 542, COMP SCI 552, COMP SCI 558, COMP SCI 559, COMP SCI 564, COMP SCI 567, COMP SCI 570, COMP SCI 576, COMP SCI 577, COMP SCI 579, COMP SCI 611, COMP SCI 612, COMP SCI 638, COMP SCI 639, COMP SCI 640, COMP SCI 642, COMP SCI 679, COMP SCI 681, COMP SCI 682, COMP SCI 699, MATH 112, MATH 113, MATH 114, MATH 130, MATH 131, MATH 132, MATH 135, MATH 141, MATH 207, MATH 211, MATH 213, MATH 217, MATH 221, MATH 222, MATH 228, MATH 234

In [ ]:
show(IOContext(stdout, :limit => false), "text/plain", cls_data[Symbol("COMP SCI 524")]);
# 
# join(keys(cls_data[Symbol("COMP SCI 524")]), " ")

Dict{String,Any} with 44 entries:
  "honors" => nothing
  "allCrossListedSubjects" => Any[Dict{String,Any}("departmentURI"=>"http://www.cs.wisc.edu/","footnotes"=>Any["Courses taught and managed by the Computer Sciences department often have enrollment restrictions that give students in UW-Madison Computer Sciences programs priority access during initial enrollment periods.\n\nEvening exams are likely for most of our undergraduate courses."],"formalDescription"=>"COMPUTER SCIENCES","undergraduateCatalogURI"=>"http://guide.wisc.edu/undergraduate/letters-science/computer-sciences/","termCode"=>"1214","departmentOwnerAcademicOrgCode"=>"L0780","description"=>"COMPUTER SCIENCES","graduateCatalogURI"=>"http://guide.wisc.edu/graduate/computer-sciences/","uddsFundingSource"=>"A4820","shortDescription"=>"COMP SCI","subjectCode"=>"266","schoolCollege"=>Dict{String,Any}("schoolCollegeURI"=>"http://www.ls.wisc.edu/","shortDescription"=>"Letters and Science","formalDescription"=>"Letters and Scienc

In [211]:
# for (k, v) in sort(cls_data)
#     println(k, ": ", v["enrollmentPrerequisites"])
# end

# Constructing the Optimization Problem

In [21]:
using JuMP, Cbc

max_credit = 6
n_sem = 6
T = 1:n_sem
C = cls

function credit(c)
    return cls_data[c]["minimumCredits"]
end

function make_cs_symbol(c)
    return Symbol("COMP SCI ", c)
end

function cs_lst(arr...)
    res = [make_cs_symbol(c) for c in arr]
    return [c for c in res if c in C]
end

function add_cs_prereq(c, prereq...)
    for t in T
        @constraint(m, 
            x[make_cs_symbol(c), t] <= sum(x[make_cs_symbol(p), i] for i in 1:t-1 for p in prereq)
        )
    end
end

function add_all_prereq()
    add_cs_prereq(300, 200)
    add_cs_prereq(354, 252)
    add_cs_prereq(354, 300)

    add_cs_prereq(506, 400)
    add_cs_prereq(506, 407, 536, 537, 559, 564, 570, 679, 552)

    add_cs_prereq(552, 352)
    add_cs_prereq(552, 354)

    for c in [400, 407, 513, 514, 534, 540, 570]
        add_cs_prereq(c, 300)
    end

    for c in [536, 537, 538, 564]
        add_cs_prereq(c, 354)
        add_cs_prereq(c, 400)
    end

    for c in [520, 577]
        add_cs_prereq(c, 400)
        add_cs_prereq(c, 240, 475)
    end

    for c in [640, 642]
        add_cs_prereq(c, 537)
    end
end

function add_cs_grad_req()
    C_basic = cs_lst(240, 252, 300, 354, 400)
    C_theory = cs_lst(577, 520)
    C_xware = cs_lst(407, 506, 536, 538, 537, 552, 564, 640, 642)
    C_app = cs_lst(412, 425, 513, 514, 524, 525, 534, 540, 545, 547, 559, 570)
    C_elec = cs_lst(407, 412, 425, 435, 471, 475, 506, 513, 514, 520, 524, 525, 526, 532, 533, 534, 536, 537, 538, 539, 540, 545, 547, 552, 558, 559, 564, 567, 570, 576, 577, 579, 635, 640, 642, 679, 639)

    
    # Take all from basic computer sciences
    for c in C_basic
        @constraint(m, sum(x[c, t] for t in T) >= 1)
    end

    # Complete 1 for Theory of computer science
    @constraint(m, sum(x[c, t] for t in T for c in C_theory) >= 1)

    # Complete 2 for Software & Hardware
    @constraint(m, sum(x[c, t] for t in T for c in C_xware) >= 2)

    # Complete 1 for Applications
    @constraint(m, sum(x[c, t] for t in T for c in C_app) >= 1)

    # Complete 2 for Electives
    @constraint(m, sum(x[c, t] for t in T for c in C_elec) >= 2)
end

function print_schedule(x)
    for t in T
        for c in C
            if x[c, t] > 0
                println("Semester ", t, ": take \"", c, ": ", cls_data[c]["title"], "\"")
            end
        end
    end
end

# m = Model(Cbc.Optimizer)
m = Model(with_optimizer(Cbc.Optimizer, logLevel=0))

@variable(m, x[C, T], Bin)

# Max credit constraint
for t in T
    @constraint(m, sum(x[c, t] * credit(c) for c in C) <= max_credit)
end

# Do not retake class
for c in C
    @constraint(m, sum(x[c, t] for t in T) <= 1)
end

# CS graduation requirements
add_cs_grad_req()

# Prerequisites
add_all_prereq()

@objective(m, Min, sum(x[c, t] * t for c in C, t in T))

optimize!(m)
println(termination_status(m))
x = value.(x)

print_schedule(x)

OPTIMAL
Semester 1: take "COMP SCI 200: Programming I"
Semester 1: take "COMP SCI 240: Introduction to Discrete Mathematics"
Semester 2: take "COMP SCI 252: Introduction to Computer Engineering"
Semester 2: take "COMP SCI 300: Programming II"
Semester 3: take "COMP SCI 534: Computational Photography"
Semester 3: take "COMP SCI 407: Foundations of Mobile Systems and Applications"
Semester 4: take "COMP SCI 400: Programming III"
Semester 4: take "COMP SCI 354: Machine Organization and Programming"
Semester 5: take "COMP SCI 506: Software Engineering"
Semester 5: take "COMP SCI 520: Introduction to Theory of Computing"
